In [2]:

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import os
from sklearn.model_selection import cross_validate
from math import pi

In [47]:
df = pd.read_csv('../raw_data/datos.csv', delimiter = ";")

/var/folders/jt/ljcpcwg95nv566n6vzs4822m0000gn/T/ipykernel_5700/2428196936.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/datos.csv', delimiter = ";")


In [48]:
df

,ContadorBajasCCC,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,diasemana,tiempo_en_empresa,edad,duracion_baja,mes_baja,epiweek
0,17664,30,2,5110,009.3,0,3187,08037,224,420,2,0,7,0.51,27.2,2,12,51
1,17664,5,2,5110,787.01,0,3187,39006,12,420,2,0,4,1.41,31.8,1,12,52
2,17664,17,2,5110,462,0,3187,38360,46,420,2,0,4,0.54,20.5,1,12,52
3,17664,16,2,5110,460,0,3187,08830,49,100,2,0,4,2.53,30.6,3,12,52
4,17664,7,1,5110,462,0,3187,02001,548,189,1,0,5,6.16,32.4,2,12,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257755,41,9,2,8121,784.0,0,8,08020,21,200,10,0,7,1.99,44.9,1,1,53
257756,36,13,2,9319,525.8,0,0,08901,223,289,7,1,7,9.68,45.1,1,1,53
257757,9,13,2,5630,525.8,0,0,08901,223,200,8,1,7,3.75,45.1,1,1,53
257758,1,1,2,9700,053.9,0,1,08028,1,200,0,0,7,4.26,36.5,1,1,53


In [110]:
app = FastAPI()

In [220]:
# Define a root `/` endpoint
@app.get('/')
def index():
    return {'greeting': 'Hello'}

In [210]:
DTYPES_RAW_OPTIMIZED = {
    "key": "O",
    "ContadorBajasCCC": "int64",
    "ContadorBajasDNI": "int64",
    "sexo": "int64",
    "cnae": "int64",
    "icd9": "float64",
    "recaida": "int64",
    "numtreb": "int64",
    "codipostal": "int64",
    "ContadordiasBajasDNI": "int64",
    "contracte": "int64",
    "grupcoti": "int64",
    "pluriempleo": "int64",
    "diasemana": "int64",
    "tiempo_en_empresa": "float64",
    "edad": "float64",
    "mes_baja": "int64",
    "epiweek": "int64"
}


In [211]:
COLUMN_NAMES_RAW = DTYPES_RAW_OPTIMIZED.keys()

In [212]:
COLUMN_NAMES_RAW

dict_keys(['key', 'ContadorBajasCCC', 'ContadorBajasDNI', 'sexo', 'cnae', 'icd9', 'recaida', 'numtreb', 'codipostal', 'ContadordiasBajasDNI', 'contracte', 'grupcoti', 'pluriempleo', 'diasemana', 'tiempo_en_empresa', 'edad', 'mes_baja', 'epiweek'])

In [218]:
# Define a prediction `/` endpoint
@app.get('/predict')
def predict(ContadorBajasCCC: int,
            ContadorBajasDNI: int,
            sexo: int,
            cnae: int,
            icd9: float,
            recaida: int,
            numtreb: int,
            codipostal: int,
            ContadordiasBajasDNI: int,
            contracte: int,
            grupcoti: int,
            pluriempleo: int,
            diasemana: int,
            tiempo_en_empresa: float,
            edad: float,
            mes_baja: int,
            epiweek: int):
    """
    we use type hinting to indicate the data types expected
    for the parameters of the function
    FastAPI uses this information in order to hand errors
    to the developpers providing incompatible parameters
    FastAPI also provides variables of the expected data type to use
    without type hinting we need to manually convert
    the parameters of the functions which are all received as strings
    """
    key = ContadorBajasCCC
    X_new = pd.DataFrame([key, ContadorBajasCCC, ContadorBajasDNI, sexo, cnae, icd9,
       recaida, numtreb, codipostal, ContadordiasBajasDNI,
       contracte, grupcoti, pluriempleo, diasemana,
       tiempo_en_empresa, edad, mes_baja, epiweek]).T
    X_new.columns = COLUMN_NAMES_RAW
    X_new = X_new.astype(DTYPES_RAW_OPTIMIZED)
    # X_new = preprocess_features(X_new)
    y_pred = float(model.predict(X_new))
    
    return {'fare_amount': y_pred}

In [20]:
df = pd.read_csv('~/code/jaimesoroa/smarthealing/raw_data/datos.csv', delimiter = ";")
df = df.drop_duplicates()
index1 = df[ (df['numtreb'] == 0)].index
df.drop(index1 , inplace=True)
index2 = df[  (df['contracte'] == '-') ].index
df.drop(index2 , inplace=True)
index3 = df[ (df['cnae'] == 0) ].index
df.drop(index3 , inplace=True)
index4 = df[  (df['codipostal'] == '-')].index
df.drop(index4 , inplace=True)
index5 = df[ (df['tiempo_en_empresa'] == 0.00)].index
df.drop(index5 , inplace=True)
index6 = df[ (df['grupcoti'] == '-')].index
df.drop(index6 , inplace=True)
df_2 = df.sort_values('icd9').head(50000)
df_2['icd9'] = pd.to_numeric(df_2['icd9'])
df_2['codipostal'] = pd.to_numeric(df_2['codipostal'])
df_2['contracte'] = pd.to_numeric(df_2['contracte'])
df_2['grupcoti'] = pd.to_numeric(df_2['grupcoti'])
X = df_2.drop(columns = ['duracion_baja'])
y = df_2["duracion_baja"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train = X.head(30000)
y_train = y.head(30000)
X_test = X.tail(20000)

/var/folders/jt/ljcpcwg95nv566n6vzs4822m0000gn/T/ipykernel_5700/2821862080.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('~/code/jaimesoroa/smarthealing/raw_data/datos.csv', delimiter = ";")


In [6]:
model = LinearRegression()
model_fit = model.fit(X_train, y_train)

In [7]:
pred_2 = model_fit.predict(X_test.iloc[[0]])
pred_2

array([67.17282546])

In [12]:
X_test.iloc[[0]]

,ContadorBajasCCC,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,diasemana,tiempo_en_empresa,edad,mes_baja,epiweek
217511,1,1,2,8623,227.3,0,1,8037,1,0,8,0,2,13.32,64.9,7,30


In [13]:
pred_3 = model_fit.predict(X_test.iloc[[50]])
pred_3

array([63.68743483])

In [14]:
X_test.iloc[[50]]

,ContadorBajasCCC,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,diasemana,tiempo_en_empresa,edad,mes_baja,epiweek
8709,17664,1,1,5110,232.0,0,3187,8860,1,100,7,0,6,13.67,39.9,11,47


In [42]:
model_LR_1 = LinearRegression()
cv_results_LR_1 = cross_validate(model_LR_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_LR_1) # Cross validation output

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
0,0.011952,0.012033,-330.174212,-1.007616,-7.378042,-159.000230,-12.609529
1,0.036457,0.008892,-352.698351,-1.149321,-7.182046,-160.197013,-12.656896
2,0.050165,0.022813,-341.333054,-1.465534,-8.001594,-174.614955,-13.214195
3,0.031806,0.012957,-326.630370,-0.238192,-16.303022,-685.122438,-26.174844
4,0.055943,0.018762,-527.093174,-0.031806,-36.834859,-7246.250563,-85.124912


In [1]:
import pandas as pd
import numpy as np
from math import pi
import os

# path = os.path.join

# 0. Load Data and Drop Duplicates

data = pd.read_csv('../raw_data/datos.csv', delimiter = ';', low_memory = False)
data.drop_duplicates(inplace = True)

# 1. Handle Null Values
# CNAE (only 482)
data = data.query('cnae != 0')

# CodiPostal (Most Frequent)
data['codipostal'] = data['codipostal'].replace('-', (data['codipostal']).mode()[0])

# Contracte (Delete few wrongs and then Most Frequent)
data = data[data['contracte'] != '001']
data = data[data['contracte'] != '019']

mf_contracte = data['contracte'].mode()[0]
data['contracte'] = data['contracte'].replace('000', mf_contracte).replace('-', mf_contracte)

# Grupcoti (Most Frequent)
mf_grupcoti = data['grupcoti'].mode()[0]
data['grupcoti'] = data['grupcoti'].replace('-', mf_grupcoti).replace('0', mf_grupcoti)

# 2. Data Cleaning
# ContadorBajasCCC (non-sense)
data.drop('ContadorBajasCCC', axis = 1, inplace = True)
# ContadorBajasDNI vs ContadordiasBajasDNI
data = data.query('ContadorBajasDNI <= ContadordiasBajasDNI')
# Categorize CNAE
c = pd.read_excel('Tabla_cnae.xlsx', usecols = ['GRUPO', 'COD_CNAE2009'])
c.rename(columns = {'GRUPO': 'cnae_cat', 'COD_CNAE2009': 'cnae'}, inplace = True)
c['cnae_cat'] = c['cnae_cat'].map(lambda x: ord(x) - 64)
c = c.query('cnae.str.isnumeric()').astype('int')
data = data.merge(c, how = 'left', on = 'cnae')

# Sickness Types
block_ends = [140,240,280,290,320,390,460,520,580,630,680,710,740,760,780,800,1000]
blocks = [[i+1, s] for i, s in enumerate(block_ends)]
# special_cases = ['E','V','M']
def make_icd9_cat(row):
    first_str = row.icd9[0]
    if first_str == 'E':
        return 18
    if first_str == 'V':
        return 19
    if first_str == 'M':
        return 20
    else:
        code = float(row.icd9)
        for block in blocks:
            if code < block[1]:
                return block[0]
data['icd9_cat'] = data.apply(lambda row: make_icd9_cat(row), axis=1)
# Sickness Wrong Values
data = data.query('sexo != 1 | icd9_cat != 11')
data = data[~((data['sexo'] == 1) & (data['icd9'].str[:2].isin(['61', '62'])))]
data = data[~((data['sexo'] == 2) & (data['icd9'].str[:2] == '60'))]
# Time in company
# Add 0.01 to have the upper limit. (0.00 means up to 3 days)
data['proporcion_baja'] = (data['duracion_baja'] / (365 * (data['tiempo_en_empresa'] + 0.01)))
by_wrong_proportion = data.sort_values(by = 'proporcion_baja', axis = 0, ascending = False)
data = data.query('proporcion_baja <= 1')
data.drop('proporcion_baja', axis = 1, inplace = True)
# Month not required
data.drop('mes_baja', axis = 1, inplace = True)
# Cyclic Calendar
data['time'] = (2 * pi * (data['epiweek'] - 1) + ((data['diasemana'] - 1) / 7)) / 52
data['sin_time'] = np.sin(data['time'])
data['cos_time'] = np.cos(data['time'])
data.drop(['time', 'diasemana', 'epiweek'], axis = 1, inplace = True)
# Categorise postal code with the first 2 numbers
data['codipostal_cat'] = data['codipostal'].str[:2]
    

/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
data['contracte'] = pd.to_numeric(data['contracte'])
data['grupcoti'] = pd.to_numeric(data['grupcoti'])
data['codipostal_cat'] = pd.to_numeric(data['codipostal_cat'])


In [5]:
data

,ContadorBajasDNI,sexo,cnae,icd9,recaida,numtreb,codipostal,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,tiempo_en_empresa,edad,duracion_baja,cnae_cat,icd9_cat,sin_time,cos_time,codipostal_cat
0,30,2,5110,009.3,0,3187,08037,224,420,2,0,0.51,27.2,2,8,1,-0.223279,0.974754,8
1,5,2,5110,787.01,0,3187,39006,12,420,2,0,1.41,31.8,1,8,16,-0.112351,0.993669,39
2,17,2,5110,462,0,3187,38360,46,420,2,0,0.54,20.5,1,8,8,-0.112351,0.993669,38
3,16,2,5110,460,0,3187,08830,49,100,2,0,2.53,30.6,3,8,8,-0.112351,0.993669,8
4,7,1,5110,462,0,3187,02001,548,189,1,0,6.16,32.4,2,8,8,-0.109621,0.993973,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248855,9,2,8121,784.0,0,8,08020,21,200,10,0,1.99,44.9,1,14,16,0.016483,0.999864,8
248856,13,2,9319,525.8,0,0,08901,223,289,7,1,9.68,45.1,1,18,9,0.016483,0.999864,8
248857,13,2,5630,525.8,0,0,08901,223,200,8,1,3.75,45.1,1,9,9,0.016483,0.999864,8
248858,1,2,9700,053.9,0,1,08028,1,200,8,0,4.26,36.5,1,20,1,0.016483,0.999864,8


In [6]:
data = data.drop(columns = 'codipostal')
data = data.drop(columns = 'icd9')
data = data.drop(columns = 'cnae')

In [7]:
data

,ContadorBajasDNI,sexo,recaida,numtreb,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,tiempo_en_empresa,edad,duracion_baja,cnae_cat,icd9_cat,sin_time,cos_time,codipostal_cat
0,30,2,0,3187,224,420,2,0,0.51,27.2,2,8,1,-0.223279,0.974754,8
1,5,2,0,3187,12,420,2,0,1.41,31.8,1,8,16,-0.112351,0.993669,39
2,17,2,0,3187,46,420,2,0,0.54,20.5,1,8,8,-0.112351,0.993669,38
3,16,2,0,3187,49,100,2,0,2.53,30.6,3,8,8,-0.112351,0.993669,8
4,7,1,0,3187,548,189,1,0,6.16,32.4,2,8,8,-0.109621,0.993973,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248855,9,2,0,8,21,200,10,0,1.99,44.9,1,14,16,0.016483,0.999864,8
248856,13,2,0,0,223,289,7,1,9.68,45.1,1,18,9,0.016483,0.999864,8
248857,13,2,0,0,223,200,8,1,3.75,45.1,1,9,9,0.016483,0.999864,8
248858,1,2,0,1,1,200,8,0,4.26,36.5,1,20,1,0.016483,0.999864,8


In [8]:
y = data['duracion_baja']

In [9]:
X = data.drop(columns = 'duracion_baja')

In [10]:
X

,ContadorBajasDNI,sexo,recaida,numtreb,ContadordiasBajasDNI,contracte,grupcoti,pluriempleo,tiempo_en_empresa,edad,cnae_cat,icd9_cat,sin_time,cos_time,codipostal_cat
0,30,2,0,3187,224,420,2,0,0.51,27.2,8,1,-0.223279,0.974754,8
1,5,2,0,3187,12,420,2,0,1.41,31.8,8,16,-0.112351,0.993669,39
2,17,2,0,3187,46,420,2,0,0.54,20.5,8,8,-0.112351,0.993669,38
3,16,2,0,3187,49,100,2,0,2.53,30.6,8,8,-0.112351,0.993669,8
4,7,1,0,3187,548,189,1,0,6.16,32.4,8,8,-0.109621,0.993973,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248855,9,2,0,8,21,200,10,0,1.99,44.9,14,16,0.016483,0.999864,8
248856,13,2,0,0,223,289,7,1,9.68,45.1,18,9,0.016483,0.999864,8
248857,13,2,0,0,223,200,8,1,3.75,45.1,9,9,0.016483,0.999864,8
248858,1,2,0,1,1,200,8,0,4.26,36.5,20,1,0.016483,0.999864,8


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169836 entries, 202966 to 51361
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ContadorBajasDNI      169836 non-null  int64  
 1   sexo                  169836 non-null  int64  
 2   recaida               169836 non-null  int64  
 3   numtreb               169836 non-null  int64  
 4   ContadordiasBajasDNI  169836 non-null  int64  
 5   contracte             169836 non-null  int64  
 6   grupcoti              169836 non-null  int64  
 7   pluriempleo           169836 non-null  int64  
 8   tiempo_en_empresa     169836 non-null  float64
 9   edad                  169836 non-null  float64
 10  cnae_cat              169836 non-null  int64  
 11  icd9_cat              169836 non-null  int64  
 12  sin_time              169836 non-null  float64
 13  cos_time              169836 non-null  float64
 14  codipostal_cat        169836 non-null  int64  
d

In [13]:
model_LR_1 = LinearRegression()
cv_results_LR_1 = cross_validate(model_LR_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_LR_1) # Cross validation output

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
0,0.049438,0.014835,-547.705172,0.128270,-25.869583,-2193.075208,-46.830281
1,0.070137,0.004791,-735.907754,0.116720,-25.753642,-2209.951816,-47.010125
2,0.086816,0.021544,-512.265190,0.127133,-25.723962,-2189.436727,-46.791417
3,0.070236,0.012567,-546.173508,0.127539,-26.139949,-2277.558907,-47.723777
4,0.103822,0.009051,-522.791413,0.130466,-25.793874,-2158.893539,-46.463895


In [96]:
from sklearn.linear_model import PoissonRegressor
model_poi_1 = PoissonRegressor()
cv_results_poi_1 = cross_validate(model_poi_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_poi_1) # Cross validation output

/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per_sample + l2_reg_strength * weights
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per_sample + l2_reg_strength * weights
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per_sample + l2_reg_strength * weights
/Users/jaimesoroarubio/.pyenv/versions/3.10.6/envs/smarthealing/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_per

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
0,0.097860,0.002138,-521.427246,-0.000185,-28.069288,-2441.787802,-49.414449
1,0.034661,0.002191,-521.523843,-0.000014,-28.379759,-2505.434396,-50.054314
2,0.047982,0.003803,-626.626714,-0.000041,-28.454341,-2579.250158,-50.786319
3,0.052806,0.008268,-521.573376,-0.000001,-28.401834,-2539.866245,-50.397086
4,0.045452,0.004680,-680.657148,-0.000085,-28.823280,-2666.393077,-51.637129


In [97]:
from sklearn.linear_model import SGDRegressor
model_SGD_1 = SGDRegressor()
cv_results_SGD_1 = cross_validate(model_SGD_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_SGD_1) # Cross validation output

,fit_time,score_time,test_max_error,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
0,2.232297,0.002342,-4.916041e+14,-3.552563e+24,-4.797758e+13,-8.673001e+27,-9.312895e+13
1,2.028665,0.002234,-5.961489e+13,-5.224722e+22,-8.469483e+12,-1.309001e+26,-1.144116e+13
2,2.028556,0.001945,-7.742209e+14,-1.107735e+25,-5.989950e+13,-2.857007e+28,-1.690268e+14
3,2.224511,0.011934,-1.330444e+14,-3.322628e+23,-1.067645e+13,-8.439020e+26,-2.904999e+13
4,1.468900,0.002511,-9.231112e+14,-1.594831e+25,-8.199371e+13,-4.252083e+28,-2.062058e+14


In [98]:
from sklearn.svm import SVR
model_SVR_1 = SVR(epsilon=0.1, C=1, kernel='linear')
cv_results_SVR_1 = cross_validate(model_SVR_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_SVR_1)

In [15]:
from sklearn.ensemble import RandomForestRegressor
model_RF_1 = RandomForestRegressor()
cv_results_RF_1 = cross_validate(model_RF_1, X_train, y_train, cv=5,
                            scoring=['max_error',
                                     'r2',
                                     'neg_mean_absolute_error',
                                     'neg_mean_squared_error',
                                     'neg_root_mean_squared_error']
) 
pd.DataFrame(cv_results_RF_1)

KeyboardInterrupt: 